In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
nltk.download('punkt')
from google.colab import files

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
uploaded = files.upload()

In [ ]:
col_names=['ID','Entity','Sentiment','Content']

In [ ]:
import io
df = pd.read_csv('twitter_training.csv',names=col_names)
df.head()

In [ ]:
df.info()


In [ ]:
df.shape()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(subset=['Content'],inplace=True)

In [ ]:
# duplicate rows
df.drop_duplicates(keep='first',inplace=True)


In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
#replacing irrelavent to beuteal
df['Sentiment']=df['Sentiment'].replace('Irrelevant','Neutral')

In [ ]:
df

EDA

In [ ]:
df.Sentiment.unique()

In [ ]:
#calculate the distribution
sentiment_distribution=df['Sentiment'].value_counts()
sentiment_distribution

In [ ]:
#create a pie chart
plt.figure(figsize=(5,5))
plt.pie(sentiment_distribution, labels=sentiment_distribution.keys())
plt.axis('equal')
plt.show()

NLP -Natural language processing


In [ ]:
# lets Load NLTK stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [ ]:
def preprocess_text(text):
    # Remove special characters, numbers, and links
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()  # Convert to lowercase
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatize words
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

# Apply preprocessing with lemmatization and stopwords removal to the 'Content' column
df['Cleaned_Content'] = df['Content'].apply(preprocess_text)

In [ ]:
# loading vader to perform sentiment analysis
nltk.download('vader_lexicon')

# Initialize the sentiment analyzer without specifying the lexicon file
sia = SentimentIntensityAnalyzer()

# Function to get sentiment scores
def get_sentiment_scores(text):
    sentiment = sia.polarity_scores(text)
    return sentiment['compound']

# Apply sentiment analysis to the 'Cleaned_Content' column
df['Sentiment_Score'] = df['Cleaned_Content'].apply(get_sentiment_scores)

In [ ]:
#Categorizing based on the score
def categorize_sentiment(score):
     if score>=0.05:
         return 'Positive'
     elif score <=-0.05:
         return 'Negative'
     else:
         return 'Neutral'

df['Sentiment_Predicted']=df['Sentiment_Score'].apply(categorize_sentiment)

In [ ]:
# Finding the distribution of the predicted sentiments
sentiment_distribution=df['Sentiment_Predicted'].value_counts()

# Creating a chart
plt.figure(figsize=(6,6))
plt.bar(sentiment_distribution.index,sentiment_distribution.values,color='pink')
plt.title('sentiment distribution')